In [1]:
import sympy as sym
import matplotlib.pyplot as plt
import os
import numpy as np
import pickle

In [15]:
S_c, I_c, I_d = sym.symbols('S_c I_c I_d')
beta, alpha, gamma, N = sym.symbols('beta alpha gamma N')
sigma_d, sigma_c = sym.symbols('sigma_d sigma_c')
r_cc, r_cd, r_dc, r_dd = sym.symbols('r_cc, r_cd, r_dc, r_dd')
s_cc, s_cd, s_dc, s_dd = sym.symbols('s_cc, s_cd, s_dc, s_dd')
t_cc, t_cd, t_dc, t_dd = sym.symbols('t_cc, t_cd, t_dc, t_dd')
p_cc, p_cd, p_dc, p_dd = sym.symbols('p_cc, p_cd, p_dc, p_dd')


S_d = 1 - S_c - I_c - I_d

f_sc = r_cc*S_c + r_cd*S_d + (s_cc - sigma_c*(S_c + S_d))*I_c + (s_cd - sigma_c*(S_c + S_d))*I_d
f_sd = r_dc*S_c + r_dd*S_d + (s_dc - sigma_d*(I_c + I_d))*I_c + (s_dd - sigma_d*(I_c + I_d))*I_d
f_ic = (t_cc - sigma_c*(S_c + S_d))*S_c + (t_cd - sigma_c*(S_c+S_d))*S_d + p_cc*I_c + p_cd*I_d
f_id = (t_dc - sigma_d*(I_c + I_d))*S_c + (t_dd - sigma_d*(I_c+I_d))*S_d + p_dc*I_c + p_dd*I_d

fbar_s = f_sc*S_c/(S_c + S_d) + f_sd*S_d/(S_c + S_d) 
fbar_i = f_ic*I_c/(I_c + I_d) + f_id*I_d/(I_c + I_d)

dScdt = -beta*S_c*(alpha*I_c + I_d) + gamma*I_c + S_c*(f_sc - fbar_s)
dIcdt = beta*S_c*(alpha*I_c + I_d) - gamma*I_c + I_c*(f_ic - fbar_i)
dSddt = -beta*S_d*(alpha*I_c + I_d) + gamma*I_d + S_d*(f_sd - fbar_s)
dIddt = beta*S_d*(alpha*I_c + I_d) - gamma*I_d + I_d*(f_id - fbar_i)

F = sym.Matrix([beta*S_c*(I_d + alpha*I_c), beta*S_d*(I_d + alpha*I_c)])
V = sym.Matrix([gamma*I_c - I_c*(f_ic - fbar_i), gamma*I_d - I_d*(f_id - fbar_i)])

dScdt_DFE = dScdt.subs([(I_c, 0), (I_d, 0)])
dSddt_DFE = dSddt.subs([(I_c, 0), (I_d, 0)]) 

SS_DFE = sym.solve(dScdt_DFE, S_c)

print(f'Flujos de entrada a la enfermedad:')
sym.print_latex(F)
print('\n')
print(f'Flujos dentro de la enfermedad y de salidad:')
sym.print_latex(V)



Flujos de entrada a la enfermedad:
\left[\begin{matrix}S_{c} \beta \left(I_{c} \alpha + I_{d}\right)\\\beta \left(I_{c} \alpha + I_{d}\right) \left(- I_{c} - I_{d} - S_{c} + 1\right)\end{matrix}\right]


Flujos dentro de la enfermedad y de salidad:
\left[\begin{matrix}I_{c} \gamma - I_{c} \left(I_{c} p_{cc} - \frac{I_{c} \left(I_{c} p_{cc} + I_{d} p_{cd} + S_{c} \left(- \sigma_{c} \left(- I_{c} - I_{d} + 1\right) + t_{cc}\right) + \left(- \sigma_{c} \left(- I_{c} - I_{d} + 1\right) + t_{cd}\right) \left(- I_{c} - I_{d} - S_{c} + 1\right)\right)}{I_{c} + I_{d}} + I_{d} p_{cd} - \frac{I_{d} \left(I_{c} p_{dc} + I_{d} p_{dd} + S_{c} \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dc}\right) + \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dd}\right) \left(- I_{c} - I_{d} - S_{c} + 1\right)\right)}{I_{c} + I_{d}} + S_{c} \left(- \sigma_{c} \left(- I_{c} - I_{d} + 1\right) + t_{cc}\right) + \left(- \sigma_{c} \left(- I_{c} - I_{d} + 1\right) + t_{cd}\right) \left(- I_{c} - I_{d} - S_

[0, 1, (-r_cd + r_dd)/(r_cc - r_cd - r_dc + r_dd)]

Flujos de entrada a los cajones de la enfermedad:
$$ F = \left[\begin{matrix}S_{c} \beta \left(I_{c} \alpha + I_{d}\right)\\\beta \left(I_{c} \alpha + I_{d}\right) \left(- I_{c} - I_{d} - S_{c} + 1\right)\end{matrix}\right]
$$ 

Flujos de salida y movimientos dentro de la enfermedad:
$$ V = \left[\begin{matrix}I_{c} \gamma - I_{c} \left(I_{c} p_{cc} - \frac{I_{c} \left(I_{c} p_{cc} + I_{d} p_{cd} + S_{c} \left(- \sigma_{c} \left(- I_{c} - I_{d} + 1\right) + t_{cc}\right) + \left(- \sigma_{c} \left(- I_{c} - I_{d} + 1\right) + t_{cd}\right) \left(- I_{c} - I_{d} - S_{c} + 1\right)\right)}{I_{c} + I_{d}} + I_{d} p_{cd} - \frac{I_{d} \left(I_{c} p_{dc} + I_{d} p_{dd} + S_{c} \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dc}\right) + \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dd}\right) \left(- I_{c} - I_{d} - S_{c} + 1\right)\right)}{I_{c} + I_{d}} + S_{c} \left(- \sigma_{c} \left(- I_{c} - I_{d} + 1\right) + t_{cc}\right) + \left(- \sigma_{c} \left(- I_{c} - I_{d} + 1\right) + t_{cd}\right) \left(- I_{c} - I_{d} - S_{c} + 1\right)\right)\\I_{d} \gamma - I_{d} \left(I_{c} p_{dc} - \frac{I_{c} \left(I_{c} p_{cc} + I_{d} p_{cd} + S_{c} \left(- \sigma_{c} \left(- I_{c} - I_{d} + 1\right) + t_{cc}\right) + \left(- \sigma_{c} \left(- I_{c} - I_{d} + 1\right) + t_{cd}\right) \left(- I_{c} - I_{d} - S_{c} + 1\right)\right)}{I_{c} + I_{d}} + I_{d} p_{dd} - \frac{I_{d} \left(I_{c} p_{dc} + I_{d} p_{dd} + S_{c} \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dc}\right) + \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dd}\right) \left(- I_{c} - I_{d} - S_{c} + 1\right)\right)}{I_{c} + I_{d}} + S_{c} \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dc}\right) + \left(- \sigma_{d} \left(I_{c} + I_{d}\right) + t_{dd}\right) \left(- I_{c} - I_{d} - S_{c} + 1\right)\right)\end{matrix}\right]
 $$


In [16]:
J_F = F.jacobian([I_c, I_d])
J_V = V.jacobian([I_c, I_d])

print(f'Jacobiano de los Flujos de entrada a la enfermedad:')
sym.print_latex(sym.simplify(J_F))
print('\n')
print(f'Jacobiano de los Flujos dentro de la enfermedad y de salidad:')
sym.print_latex(sym.simplify(J_V))

Jacobiano de los Flujos de entrada a la enfermedad:
\left[\begin{matrix}S_{c} \alpha \beta & S_{c} \beta\\\beta \left(- I_{c} \alpha - I_{d} - \alpha \left(I_{c} + I_{d} + S_{c} - 1\right)\right) & \beta \left(- I_{c} \alpha - I_{c} - 2 I_{d} - S_{c} + 1\right)\end{matrix}\right]


Jacobiano de los Flujos dentro de la enfermedad y de salidad:
\left[\begin{matrix}\frac{2 I_{c}^{3} I_{d} \sigma_{c} + 2 I_{c}^{3} I_{d} \sigma_{d} + 5 I_{c}^{2} I_{d}^{2} \sigma_{c} + 5 I_{c}^{2} I_{d}^{2} \sigma_{d} - I_{c}^{2} I_{d} p_{cc} + I_{c}^{2} I_{d} p_{dc} - 2 I_{c}^{2} I_{d} \sigma_{c} - I_{c}^{2} I_{d} \sigma_{d} + I_{c}^{2} I_{d} t_{cd} - I_{c}^{2} I_{d} t_{dd} + I_{c}^{2} \gamma + 4 I_{c} I_{d}^{3} \sigma_{c} + 4 I_{c} I_{d}^{3} \sigma_{d} - 2 I_{c} I_{d}^{2} p_{cc} + 2 I_{c} I_{d}^{2} p_{dc} - 4 I_{c} I_{d}^{2} \sigma_{c} - 2 I_{c} I_{d}^{2} \sigma_{d} + 2 I_{c} I_{d}^{2} t_{cd} - 2 I_{c} I_{d}^{2} t_{dd} + 2 I_{c} I_{d} \gamma + I_{d}^{4} \sigma_{c} + I_{d}^{4} \sigma_{d} - I_{d}^{3} p_{cd} 

$$ J_{F} = \left[\begin{matrix}S_{c} \alpha \beta & S_{c} \beta\\\beta \left(- I_{c} \alpha - I_{d} - \alpha \left(I_{c} + I_{d} + S_{c} - 1\right)\right) & \beta \left(- I_{c} \alpha - I_{c} - 2 I_{d} - S_{c} + 1\right)\end{matrix}\right] $$

$$ J_{V} = \left[\begin{matrix}\frac{2 I_{c}^{3} I_{d} \sigma_{c} + 2 I_{c}^{3} I_{d} \sigma_{d} + 5 I_{c}^{2} I_{d}^{2} \sigma_{c} + 5 I_{c}^{2} I_{d}^{2} \sigma_{d} - I_{c}^{2} I_{d} p_{cc} + I_{c}^{2} I_{d} p_{dc} - 2 I_{c}^{2} I_{d} \sigma_{c} - I_{c}^{2} I_{d} \sigma_{d} + I_{c}^{2} I_{d} t_{cd} - I_{c}^{2} I_{d} t_{dd} + I_{c}^{2} \gamma + 4 I_{c} I_{d}^{3} \sigma_{c} + 4 I_{c} I_{d}^{3} \sigma_{d} - 2 I_{c} I_{d}^{2} p_{cc} + 2 I_{c} I_{d}^{2} p_{dc} - 4 I_{c} I_{d}^{2} \sigma_{c} - 2 I_{c} I_{d}^{2} \sigma_{d} + 2 I_{c} I_{d}^{2} t_{cd} - 2 I_{c} I_{d}^{2} t_{dd} + 2 I_{c} I_{d} \gamma + I_{d}^{4} \sigma_{c} + I_{d}^{4} \sigma_{d} - I_{d}^{3} p_{cd} + I_{d}^{3} p_{dd} - 2 I_{d}^{3} \sigma_{c} - I_{d}^{3} \sigma_{d} + I_{d}^{3} t_{cd} - I_{d}^{3} t_{dd} - I_{d}^{2} S_{c} t_{cc} + I_{d}^{2} S_{c} t_{cd} + I_{d}^{2} S_{c} t_{dc} - I_{d}^{2} S_{c} t_{dd} + I_{d}^{2} \gamma + I_{d}^{2} \sigma_{c} - I_{d}^{2} t_{cd} + I_{d}^{2} t_{dd}}{I_{c}^{2} + 2 I_{c} I_{d} + I_{d}^{2}} & \frac{I_{c} \left(- I_{c} \left(I_{c} p_{cc} + I_{d} p_{cd} - S_{c} \left(\sigma_{c} \left(- I_{c} - I_{d} + 1\right) - t_{cc}\right) + \left(\sigma_{c} \left(- I_{c} - I_{d} + 1\right) - t_{cd}\right) \left(I_{c} + I_{d} + S_{c} - 1\right)\right) - I_{d} \left(I_{c} p_{dc} + I_{d} p_{dd} - S_{c} \left(\sigma_{d} \left(I_{c} + I_{d}\right) - t_{dc}\right) + \left(\sigma_{d} \left(I_{c} + I_{d}\right) - t_{dd}\right) \left(I_{c} + I_{d} + S_{c} - 1\right)\right) + \left(I_{c} + I_{d}\right)^{2} \left(- S_{c} \sigma_{c} - p_{cd} + \sigma_{c} \left(I_{c} + I_{d} - 1\right) + \sigma_{c} \left(I_{c} + I_{d} + S_{c} - 1\right) + t_{cd}\right) + \left(I_{c} + I_{d}\right) \left(I_{c} p_{dc} + I_{c} \left(S_{c} \sigma_{c} + p_{cd} - \sigma_{c} \left(I_{c} + I_{d} - 1\right) - \sigma_{c} \left(I_{c} + I_{d} + S_{c} - 1\right) - t_{cd}\right) + I_{d} p_{dd} - I_{d} \left(S_{c} \sigma_{d} - p_{dd} - \sigma_{d} \left(I_{c} + I_{d}\right) + \sigma_{d} \left(- I_{c} - I_{d} - S_{c} + 1\right) + t_{dd}\right) - S_{c} \left(\sigma_{d} \left(I_{c} + I_{d}\right) - t_{dc}\right) + \left(\sigma_{d} \left(I_{c} + I_{d}\right) - t_{dd}\right) \left(I_{c} + I_{d} + S_{c} - 1\right)\right)\right)}{\left(I_{c} + I_{d}\right)^{2}}\\\frac{I_{d} \left(- I_{c} \left(I_{c} p_{cc} + I_{d} p_{cd} - S_{c} \left(\sigma_{c} \left(- I_{c} - I_{d} + 1\right) - t_{cc}\right) + \left(\sigma_{c} \left(- I_{c} - I_{d} + 1\right) - t_{cd}\right) \left(I_{c} + I_{d} + S_{c} - 1\right)\right) - I_{d} \left(I_{c} p_{dc} + I_{d} p_{dd} - S_{c} \left(\sigma_{d} \left(I_{c} + I_{d}\right) - t_{dc}\right) + \left(\sigma_{d} \left(I_{c} + I_{d}\right) - t_{dd}\right) \left(I_{c} + I_{d} + S_{c} - 1\right)\right) + \left(I_{c} + I_{d}\right)^{2} \left(S_{c} \sigma_{d} - p_{dc} - \sigma_{d} \left(I_{c} + I_{d}\right) - \sigma_{d} \left(I_{c} + I_{d} + S_{c} - 1\right) + t_{dd}\right) + \left(I_{c} + I_{d}\right) \left(I_{c} p_{cc} + I_{c} \left(S_{c} \sigma_{c} + p_{cc} - \sigma_{c} \left(I_{c} + I_{d} - 1\right) - \sigma_{c} \left(I_{c} + I_{d} + S_{c} - 1\right) - t_{cd}\right) + I_{d} p_{cd} - I_{d} \left(S_{c} \sigma_{d} - p_{dc} - \sigma_{d} \left(I_{c} + I_{d}\right) + \sigma_{d} \left(- I_{c} - I_{d} - S_{c} + 1\right) + t_{dd}\right) - S_{c} \left(\sigma_{c} \left(- I_{c} - I_{d} + 1\right) - t_{cc}\right) + \left(\sigma_{c} \left(- I_{c} - I_{d} + 1\right) - t_{cd}\right) \left(I_{c} + I_{d} + S_{c} - 1\right)\right)\right)}{\left(I_{c} + I_{d}\right)^{2}} & \frac{- I_{c}^{4} \sigma_{c} - I_{c}^{4} \sigma_{d} - 4 I_{c}^{3} I_{d} \sigma_{c} - 4 I_{c}^{3} I_{d} \sigma_{d} + I_{c}^{3} p_{cc} - I_{c}^{3} p_{dc} + 2 I_{c}^{3} \sigma_{c} + I_{c}^{3} \sigma_{d} - I_{c}^{3} t_{cd} + I_{c}^{3} t_{dd} - 5 I_{c}^{2} I_{d}^{2} \sigma_{c} - 5 I_{c}^{2} I_{d}^{2} \sigma_{d} + 2 I_{c}^{2} I_{d} p_{cd} - 2 I_{c}^{2} I_{d} p_{dd} + 4 I_{c}^{2} I_{d} \sigma_{c} + 2 I_{c}^{2} I_{d} \sigma_{d} - 2 I_{c}^{2} I_{d} t_{cd} + 2 I_{c}^{2} I_{d} t_{dd} + I_{c}^{2} S_{c} t_{cc} - I_{c}^{2} S_{c} t_{cd} - I_{c}^{2} S_{c} t_{dc} + I_{c}^{2} S_{c} t_{dd} + I_{c}^{2} \gamma - I_{c}^{2} \sigma_{c} + I_{c}^{2} t_{cd} - I_{c}^{2} t_{dd} - 2 I_{c} I_{d}^{3} \sigma_{c} - 2 I_{c} I_{d}^{3} \sigma_{d} + I_{c} I_{d}^{2} p_{cd} - I_{c} I_{d}^{2} p_{dd} + 2 I_{c} I_{d}^{2} \sigma_{c} + I_{c} I_{d}^{2} \sigma_{d} - I_{c} I_{d}^{2} t_{cd} + I_{c} I_{d}^{2} t_{dd} + 2 I_{c} I_{d} \gamma + I_{d}^{2} \gamma}{I_{c}^{2} + 2 I_{c} I_{d} + I_{d}^{2}}\end{matrix}\right]
 $$




In [17]:
J_F = J_F.subs([(I_d, 0), (I_c, 0)])
J_V = J_V.subs([(I_d, 0), (I_c, 0)])

NGM = J_F*J_V**-1
print('Jacobiano de F en el DFE:')
sym.print_latex(J_F)

print('\nJacobiano de V en el DFE:')
sym.print_latex(J_V)

print('\nMatriz de la próxima generación (NGM):')
sym.print_latex(NGM)

Jacobiano de F en el DFE:
\left[\begin{matrix}S_{c} \alpha \beta & S_{c} \beta\\\alpha \beta \left(1 - S_{c}\right) & \beta \left(1 - S_{c}\right)\end{matrix}\right]

Jacobiano de V en el DFE:
\left[\begin{matrix}\gamma & 0\\0 & - S_{c} t_{dc} + S_{c} \left(- \sigma_{c} + t_{cc}\right) + \gamma - t_{dd} \left(1 - S_{c}\right) + \left(1 - S_{c}\right) \left(- \sigma_{c} + t_{cd}\right)\end{matrix}\right]

Matriz de la próxima generación (NGM):
\left[\begin{matrix}\frac{S_{c} \alpha \beta}{\gamma} & \frac{S_{c} \beta}{- S_{c} t_{dc} + S_{c} \left(- \sigma_{c} + t_{cc}\right) + \gamma - t_{dd} \left(1 - S_{c}\right) + \left(1 - S_{c}\right) \left(- \sigma_{c} + t_{cd}\right)}\\\frac{\alpha \beta \left(1 - S_{c}\right)}{\gamma} & \frac{\beta \left(1 - S_{c}\right)}{- S_{c} t_{dc} + S_{c} \left(- \sigma_{c} + t_{cc}\right) + \gamma - t_{dd} \left(1 - S_{c}\right) + \left(1 - S_{c}\right) \left(- \sigma_{c} + t_{cd}\right)}\end{matrix}\right]


$$ J_{F}(DFE) = \left[\begin{matrix}S_{c} \alpha \beta & S_{c} \beta\\\alpha \beta \left(1 - S_{c}\right) & \beta \left(1 - S_{c}\right)\end{matrix}\right]
 $$

$$ J_{V}(DFE) = \left[\begin{matrix}\gamma & 0\\0 & - S_{c} t_{dc} + S_{c} \left(- \sigma_{c} + t_{cc}\right) + \gamma - t_{dd} \left(1 - S_{c}\right) + \left(1 - S_{c}\right) \left(- \sigma_{c} + t_{cd}\right)\end{matrix}\right]
 $$

Matriz de la próxima generación:
$$ NGM = \left[\begin{matrix}\frac{S_{c} \alpha \beta}{\gamma} & \frac{S_{c} \beta}{- S_{c} t_{dc} + S_{c} \left(- \sigma_{c} + t_{cc}\right) + \gamma - t_{dd} \left(1 - S_{c}\right) + \left(1 - S_{c}\right) \left(- \sigma_{c} + t_{cd}\right)}\\\frac{\alpha \beta \left(1 - S_{c}\right)}{\gamma} & \frac{\beta \left(1 - S_{c}\right)}{- S_{c} t_{dc} + S_{c} \left(- \sigma_{c} + t_{cc}\right) + \gamma - t_{dd} \left(1 - S_{c}\right) + \left(1 - S_{c}\right) \left(- \sigma_{c} + t_{cd}\right)}\end{matrix}\right]
$$




In [25]:
eigvals_NGM = NGM.eigenvals()
print('Valores propios y su multiplicidad algebraica de la NGM:')
sym.print_latex(eigvals_NGM)

R0 = list(eigvals_NGM.keys())[0]


R0_ScDom = sym.simplify(R0.subs(S_c, SS_DFE[0]))
R0_ScExt = sym.simplify(R0.subs(S_c, SS_DFE[1]))
R0_ScCoex = sym.simplify(R0.subs(S_c, SS_DFE[2]))

print('\nR0 cuando toda la población es cuidadosa:')
sym.print_latex(R0_ScDom)

print('\nR0 cuando toda la población es descuidada')
sym.print_latex(R0_ScExt)

print('\nR0 cuando toda la población ')
sym.print_latex(R0_ScCoex)


Valores propios y su multiplicidad algebraica de la NGM:
\left\{ 0 : 1, \  \frac{\beta \left(S_{c}^{2} \alpha t_{cc} - S_{c}^{2} \alpha t_{cd} - S_{c}^{2} \alpha t_{dc} + S_{c}^{2} \alpha t_{dd} + S_{c} \alpha \gamma - S_{c} \alpha \sigma_{c} + S_{c} \alpha t_{cd} - S_{c} \alpha t_{dd} - S_{c} \gamma + \gamma\right)}{\gamma \left(S_{c} t_{cc} - S_{c} t_{cd} - S_{c} t_{dc} + S_{c} t_{dd} + \gamma - \sigma_{c} + t_{cd} - t_{dd}\right)} : 1\right\}

R0 cuando toda la población es cuidadosa:
\frac{\beta}{\gamma - \sigma_{c} + t_{cd} - t_{dd}}

R0 cuando toda la población es descuidada
\frac{\alpha \beta}{\gamma}

R0 cuando toda la población 
\frac{\beta \left(\alpha \left(r_{cd} - r_{dd}\right)^{2} \left(t_{cc} - t_{cd} - t_{dc} + t_{dd}\right) + \gamma \left(r_{cc} - r_{cd} - r_{dc} + r_{dd}\right)^{2} + \left(r_{cd} - r_{dd}\right) \left(r_{cc} - r_{cd} - r_{dc} + r_{dd}\right) \left(- \alpha \gamma + \alpha \sigma_{c} - \alpha t_{cd} + \alpha t_{dd} + \gamma\right)\right)}{\gamma \left(

Valores propios y su multiplicidad algebraica de la NGM:
$$ \left\{ 0 : 1, \  \frac{\beta \left(S_{c}^{2} \alpha t_{cc} - S_{c}^{2} \alpha t_{cd} - S_{c}^{2} \alpha t_{dc} + S_{c}^{2} \alpha t_{dd} + S_{c} \alpha \gamma - S_{c} \alpha \sigma_{c} + S_{c} \alpha t_{cd} - S_{c} \alpha t_{dd} - S_{c} \gamma + \gamma\right)}{\gamma \left(S_{c} t_{cc} - S_{c} t_{cd} - S_{c} t_{dc} + S_{c} t_{dd} + \gamma - \sigma_{c} + t_{cd} - t_{dd}\right)} : 1\right\}
 $$

$R0$ cuando se tiene toda la población de susceptibles en las personas cuidadosas:
$$ R0(S_{c}=1) = \frac{\beta}{\gamma - \sigma_{c} + t_{cd} - t_{dd}}
$$

$R0$ cuando se tiene toda la población de susceptibles en las personas descuidadas:
$$ R0(S_{d}=1) = \frac{\alpha \beta}{\gamma}$$

 $R0$ cuando se tiene la población de susceptibles mezclada entre cuidadosos y descuidados:
$$ R0(S_{d}=1) = \frac{\beta \left(\alpha \left(r_{cd} - r_{dd}\right)^{2} \left(t_{cc} - t_{cd} - t_{dc} + t_{dd}\right) + \gamma \left(r_{cc} - r_{cd} - r_{dc} + r_{dd}\right)^{2} + \left(r_{cd} - r_{dd}\right) \left(r_{cc} - r_{cd} - r_{dc} + r_{dd}\right) \left(- \alpha \gamma + \alpha \sigma_{c} - \alpha t_{cd} + \alpha t_{dd} + \gamma\right)\right)}{\gamma \left(r_{cc} - r_{cd} - r_{dc} + r_{dd}\right) \left(- t_{cc} \left(r_{cd} - r_{dd}\right) + t_{cd} \left(r_{cd} - r_{dd}\right) + t_{dc} \left(r_{cd} - r_{dd}\right) - t_{dd} \left(r_{cd} - r_{dd}\right) + \left(\gamma - \sigma_{c} + t_{cd} - t_{dd}\right) \left(r_{cc} - r_{cd} - r_{dc} + r_{dd}\right)\right)}
$$

In [23]:
func_R0 = sym.lambdify([S_c, S_d, beta, alpha, gamma, sigma_c, t_cc, t_cd, t_dc, t_dd], R0, 'numpy')
with open('R0.pickle', 'wb') as file:
    pickle.dump(R0, file)

In [25]:
print(func_R0(1,2,4,5,6,7,8,9,10,11))

3.2063492063492065
